In [1]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt

In [119]:
def show_image(image, title="Image"):
    """Display an image with cv2
    
    Args:
        image (ndarray): An imag to show
        title (str): A window title.
    """
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    
def transform(target, matrix):
    """Transform target image with transform matrix
    
    Args:
        target (ndarray): image to transform
        matrix (list, ndarray): Either list of 6 floats or a numpy array of shape 2,3 - transform matrix
    """
    error = ValueError("Error: matrix must be a list of 6 elements or ndarraray of shape (2,3)")
    if type(matrix) is list:
        if len(matrix) != 6:
            raise error
        matrix = np.array(matrix).reshape(2,3)
    elif not (hasattr(matrix, 'shape') and matrix.shape == (2,3)):
        raise error
    return cv2.warpAffine(target, matrix, target.shape[:2][::-1])


def cut_off(target, roi_corners):
    """Cut off a roi polygon from image
    
    Args:
        target (ndarray): target image
        roi_corners (list): list of polygon lists
        
    Returns:
        A numpy array with roi cropped to its bounding box and mask
    """
    
    mask = np.zeros(target.shape, dtype=np.uint8)
    channel_count = target.shape[2]
    ignore_mask_color = (255,)*channel_count
    roi_corners = np.array([roi_corners], dtype=np.int32)
    
    _ = cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    masked_image = cv2.bitwise_and(target, mask)
    x,y,w,h = cv2.boundingRect(mask[:,:,1])
    return masked_image[x:x+w,y:y+h,:], mask[x:x+w, y:y+h,1]


def cut_off_alt(target, roi_corners):
    """Cut off a roi polygon from image
    
    Args:
        target (ndarray): target image
        roi_corners (list): list of polygon lists
        
    Returns:
        A numpy array with roi cropped to its bounding box and mask
    """
    
    mask = np.zeros(target.shape, dtype=np.uint8)
    channel_count = target.shape[2]
    ignore_mask_color = (255,)*channel_count
    roi_corners = np.array([roi_corners], dtype=np.int32)
    
    _ = cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    masked_image = cv2.bitwise_and(target, mask)
    x,y,w,h = cv2.boundingRect(mask[:,:,1])
    return masked_image, mask[:,:,1]


def paste_pice(target, pice, mask, pos):
    """Paste pice on top of the target picture with mask.
    
    Args:
        target (ndarray): target picture
        pice (ndarray): picture to paste
        mask (ndarray): picture mask
        pos (tuple): (x, y) position on target picture to put pice on
    """
    x,y = pos
    w,h = pice.shape[:2]
    mask_inv = cv2.bitwise_not(mask)
    roi = target[x:x+w, y:y+h]
    blacked = cv2.bitwise_and(roi, roi, mask = mask_inv)
    dst = cv2.add(blacked,pice)
    target[x:x+w, y:y+h] = dst
    
    
def paste_pice_alt(target, pice, mask):
    """Paste pice on top of the target picture with mask.
    
    Args:
        target (ndarray): target picture
        pice (ndarray): picture to paste
        mask (ndarray): picture mask
        pos (tuple): (x, y) position on target picture to put pice on
    """
    mask_inv = cv2.bitwise_not(mask)
    blacked = cv2.bitwise_and(target, target, mask = mask_inv)
    target = cv2.add(blacked,pice)

In [3]:
t = cv2.imread('square.jpg')

In [4]:
show_image(t)

In [5]:
im, mask = cut_off(t, [(100,100), (100, 600), (600,600)])

In [6]:
show_image(im)

# Squared dispersion

In [382]:
target = cv2.imread('square.jpg')

In [8]:
square_size = (50, 50)
sw,sh = square_size
w,h,_ = target.shape

In [391]:
show_image(mask)

In [256]:
square = [(x0, y0), (x0, y0+sh), (x0+sw, y0+sh), (x0+sw, y0)]

In [21]:
target[x0:x0+sw,y0:y0+sh,:] = 0

In [388]:
sq, mask = cut_off_alt(target, square)

In [389]:
coef = 1/np.round(np.log(y0+1))
mask = transform(mask, [coef, 0, h/2-y0, 0, coef, 0])
sq = transform(sq,  [coef, 0, h/2-y0, 0, coef, 0])

In [386]:
mask_inv = cv2.bitwise_not(mask)
blacked = cv2.bitwise_and(target, target, mask = mask_inv)

In [379]:
t = cv2.add(blacked, sq)

In [366]:
paste_pice_alt(target, sq, mask)

In [50]:
y0 = 0
while y0<h/2:
    x0 = 0
    while x0<w-sw:
        square = [(x0, y0), (x0, y0+sh), (x0+sw, y0+sh), (x0+sw, y0)] 
        sq, mask = cut_off(target, square)
        target[x0:x0+sw,y0:y0+sh,:] = 255
        sq = transform(sq, [1/(y0+1), 0, h/2-y0, 0, 1/(y0+1), 0])
        mask = transform(mask, [1/(y0+1), 0, h/2-y0, 0, 1/(y0+1), 0])
        paste_pice(target, sq, mask, (x0, y0))
        x0 += sw
    y0 += sh

In [51]:
show_image(target)